In [13]:
'''
This script generates a JSON summary of the provided TV Viewing Figure files.

The output is a summary of the monthly viewing figures per group (Month, Channel and Producer).
Additionally, the most viewed programme per group and a cumulative totals are provided.

Context:
    STEP 1: Data Cleanse
    STEP 2: Data Processing
    STEP 3: Generate Summary
    STEP 4: Output Summary and Checks
'''

import pandas as pd
import json

## Read TV Viewing figure datasets ##

viewing_data = pd.read_excel("2024 Mediacells Python Task TV data.xlsx", sheet_name="Viewing data")
producers = pd.read_excel("2024 Mediacells Python Task TV data.xlsx", sheet_name="Producers")

In [14]:
## STEP 1: Data Cleanse ##

# Check if viewing data is complete, if missing data is found, throw error
if viewing_data.isnull().values.any() == True:
    raise ValueError("Input file contains missing data.")

# Remove leading and trailing whitespace from df using '.strip()'
viewing_data = viewing_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Standardise Datetime format
viewing_data["Date"] = pd.to_datetime(viewing_data["Date"])

In [15]:
## STEP 2: Data Processing ##

# Merge exercise dataframes
combined = viewing_data.merge(producers, on="Programme")

# Separate Date into Year and Month(name)
combined["Year"] = combined["Date"].dt.strftime("%Y")
combined["Month"] = combined["Date"].dt.strftime("%B")

# Get unique 'Programme' values from both datasets to check mismatches
view_prog = viewing_data.Programme.unique()
prod_prog = producers.Programme.unique()
# Find which Programmes were not merged between datasets
viewing_data_check, producers_check = (list(set(view_prog) - set(prod_prog)), list(set(prod_prog) - set(view_prog)))

In [16]:
## STEP 3: Generate summary ## 

# Find Monthly viewers per group (Month, Channel, Producer)
output = combined.groupby(["Year", "Month", "Channel", "Producer"])["Viewers"].sum().reset_index(name="Monthly Viewers")

## Find the 'Programme' with the highest viewing figures per group ##

# Group to ignore territory
prep_highest_viewers = combined.groupby(["Year", "Month", "Channel", "Programme", "Producer"])["Viewers"].sum().reset_index(name="Viewers")

# Using idxmax() find rows(Programmes) with highest viewing figures per group
idx = prep_highest_viewers.groupby(["Year", "Month", "Channel", "Producer"])["Viewers"].idxmax()
highest_viewers = prep_highest_viewers.loc[idx]

# Add Programme column back into df to reveal most viewed programmes and rename
highest_viewers = highest_viewers[["Year", "Month", "Channel", "Programme", "Producer"]]
highest_viewers = highest_viewers.rename(columns={"Programme": "Most Viewed Programme"})

# Merge Most Viewed Programme per group onto output df
output = output.merge(highest_viewers, on=["Year", "Month", "Channel", "Producer"])

## Add cumulative total to output ##
output["Cumulative Total"] = output["Monthly Viewers"].cumsum()

In [17]:
## STEP 4: Output Summary and Checks ##

# Output summary to JSON
result = output.to_json(orient="records")

# Check JSON is not empty and was created successfully
check_obj = json.loads(result)
if len(check_obj) != 0:
    output.to_json("stakeholder_summary.json", orient="records")
    print("Complete: Output JSON 'stakeholder_summary.json' generated successfully.\n")
else:
    raise ValueError("Output JSON empty.")


## Checks ##
print("\n## Checks ##")
if len(viewing_data_check) > 0:
    print("The Viewing dataset had programmes missing from the Producers dataset.")
    print("Programmes:")
    for item in viewing_data_check:
        print("     - "+ item)

if len(producers_check) > 0:
    print("\nThe Producers dataset had programmes missing from the Viewing dataset.")
    print("Programmes:")
    for item in producers_check:
        print("     - "+ item)

Complete: Output JSON 'stakeholder_summary.json' generated successfully.


## Checks ##
The Viewing dataset had programmes missing from the Producers dataset.
Programmes:
     - C

The Producers dataset had programmes missing from the Viewing dataset.
Programmes:
     - D
